### Training of modified U-Net for Optic Disc on RIM-ONE v3 database (cross-validation fold #0).

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt

from model_rimone import *
from process_images_rimone import *
from process_result import *

print('Keras version:', keras.__version__)
print('TensorFlow version:', tf.__version__)

In [ ]:
images, cups, disc_locations = get_images()

In [ ]:
gen = data_generator(images, cups, disc_locations, 128, 'train', batch_size=1)
batch = next(gen)
batch[0].shape

In [ ]:
fig = plt.imshow(batch[0][0])
plt.show()
plt.imshow(batch[1][0], cmap=plt.cm.Greys_r); plt.colorbar(); plt.show()

In [ ]:
X_valid, Y_valid = next(data_generator(images, cups, disc_locations, train_or_test='test', batch_size=100, stationary=True))
plt.imshow(X_valid[0]); plt.show()
plt.imshow(Y_valid[0]); plt.show()
print(X_valid.shape, Y_valid.shape)

In [ ]:
epochs = NUM_EPOCHS
spe = SPE
img_size = IMG_SIZE

model = create_compile_model(img_size, 3e-4)
history_cup = train_cup(images, 
                     cups, 
                     disc_locations,
                     'cup_color_rimone', 
                     model, epochs, X_valid, Y_valid, img_size, spe)

In [ ]:
pred_iou, pred_dice, pred_cup = predict(images, X_valid, Y_valid, model, img_size)

In [ ]:
print(np.mean(pred_iou))
print(np.mean(pred_dice))

In [ ]:
print(np.std(pred_iou))
print(np.std(pred_dice))

In [ ]:
images = h5f['RIM-ONE v3/256 px/images']
discs = h5f['RIM-ONE v3/256 px/disc']

X_valid, Z_valid = next(data_generator_disc(images, discs, train_or_test='test', batch_size=100, stationary=True))
plt.imshow(X_valid[0]); plt.show()
print(X_valid.shape, Z_valid.shape)

In [ ]:
plt.imshow(Z_valid[0]); plt.show()

In [ ]:
img_size = 256

model_disc = create_compile_model(img_size, 1e-3)
history_disc = train_disc(images, 
                     discs, 
                     'disc_color_rimone', 
                     model_disc, epochs, X_valid, Z_valid, img_size, spe)

In [ ]:
pred_iou, pred_dice, pred_disc = predict(images, X_valid, Z_valid, model_disc, img_size)

In [ ]:
print(np.mean(pred_iou))
print(np.mean(pred_dice))

In [ ]:
plot_results(history_cup, epochs)

In [ ]:
plot_results(history_disc, epochs)

In [ ]:
df, diametros = create_table_result(pred_cup, pred_disc, test_idx, 'rgb_red')

In [ ]:
print(df)

In [ ]:
print(diametros)